In [2]:
import os
import json
import statistics
import pandas as pd
from collections import Counter

In [3]:
def filter_zero_columns(dataframe): #delete columnds that are 0.0 down the entire line
    for column in dataframe:
        if sum(dataframe[column]) == 0.0:
            del dataframe[column]

def zero_intervals(l):
    result = list()
    z = 0
    for i in range(1,len(l)):
        if l[i]-l[i-1] == 0:
            z += 1
        else:
            result.append(z)
            z = 0
    if z:
        result.append(z)
    return dict(Counter(result))

def diff_list(l):
    return [ float(float(l[i]) - float(l[i-1])) for i in range(1,len(l))]

def avg_time_between_samples(ts):
    statistics.mean( diff_list(ts) )
def avg_time_between_samples(ts):
    statistics.stdev( diff_list(ts) )

def avg_energy_increase(energy):
    return statistics.mean(  diff_list(energy) )
def stdev_energy_increase(energy):
    return statistics.stdev( diff_list(energy) )

In [7]:
results_dir = 'TMP_RES/jolteon-results'
os.chdir(results_dir)
os.listdir()

['zxing_7_c-dynamicarray.metadata.json',
 'zxing_9_c-dynamicarray.csv',
 'zxing_10_c-dynamicarray.metadata.json',
 'zxing_1_c-dynamicarray.metadata.json',
 'zxing_8_c-dynamicarray.csv',
 'zxing_6_c-dynamicarray.csv',
 'zxing_6_c-dynamicarray.metadata.json',
 'zxing_9_c-dynamicarray.metadata.json',
 'zxing_7_c-dynamicarray.csv',
 'zxing_3_c-dynamicarray.metadata.json',
 'zxing_5_c-dynamicarray.csv',
 'zxing_4_c-dynamicarray.csv',
 'zxing_4_c-dynamicarray.metadata.json',
 'zxing_10_c-dynamicarray.csv',
 'zxing_2_c-dynamicarray.metadata.json',
 'zxing_8_c-dynamicarray.metadata.json',
 'zxing_3_c-dynamicarray.csv',
 'zxing_1_c-dynamicarray.csv',
 'zxing_2_c-dynamicarray.csv',
 'zxing_5_c-dynamicarray.metadata.json']

In [8]:
datafiles = os.listdir()
datafilenames = list(set([ x.split('.')[0] for x in datafiles]))
datafilenames

['zxing_7_c-dynamicarray',
 'zxing_3_c-dynamicarray',
 'zxing_10_c-dynamicarray',
 'zxing_5_c-dynamicarray',
 'zxing_2_c-dynamicarray',
 'zxing_6_c-dynamicarray',
 'zxing_4_c-dynamicarray',
 'zxing_8_c-dynamicarray',
 'zxing_1_c-dynamicarray',
 'zxing_9_c-dynamicarray']

In [9]:
## Initializes 'metadata' and 'data'

parts = datafilenames[0].split('_')
benchmark = parts[0]
iter = parts[1]
type = parts[2]

filename_base = datafilenames[0]
with open(f'{filename_base}.metadata.json') as fh: metadata = json.loads(fh.read())
data = pd.read_csv(f'{filename_base}.csv')
filter_zero_columns(data)

data

,socket,dram,pkg,timestamp
0,1,14342.5173,183488.4308,1616405984772497
1,2,62234.2950,86757.3414,1616405984772512
2,1,14342.5173,183488.4453,1616405984773637
3,2,62234.2997,86757.3636,1616405984773655
4,1,14342.5173,183488.4558,1616405984774769
...,...,...,...,...
2317,2,62238.4555,86801.7944,1616405986079169
2318,1,14343.4015,183524.5811,1616405986080278
2319,2,62238.4563,86801.8145,1616405986080293
2320,1,14343.4015,183524.5933,1616405986081402


In [33]:
result = {}
result['metadata'] = metadata

num_sockets = max(data['socket'])
for i in range(num_sockets): # filling out the socket level of result{}
    socket = i+1
    current = data[data.socket.eq(socket)].to_dict()
    del current['socket']
    result[socket] = current
    timestamps = result[socket]['timestamp']
    #print(timestamps[:10])
 
print(result)

for socket in result: # filling out the actual computations in result{}
    if socket == 'metadata': continue
    timestamps = result[socket]['timestamp']
    del result[socket]['timestamp']
    power_domains = list(result[socket])
    for powd in power_domains:
        energy = result[socket][powd]
        energy = [ energy[i] for i in energy ] #convert from dict<int,double> to list<double>
        result[socket][powd] = dict()
        #result[socket][powd]['zero-intervals'] = zero_intervals(energy)
        result[socket][powd]['nonzero-energy-increase'] = dict()
        result[socket][powd]['nonzero-energy-increase']['avg'] = avg_nonzero_energy_increase(energy)
        result[socket][powd]['nonzero-energy-increase']['stdev'] = stdev_nonzero_energy_increase(energy)

result

{'metadata': {'samplingRate': 1, 'lifetime': 1309, 'numSamples': 2322}, 1: {'dram': {0: 14342.5173, 2: 14342.5173, 4: 14342.5173, 6: 14342.5173, 8: 14342.5173, 10: 14342.5173, 12: 14342.5173, 14: 14342.5173, 16: 14342.5173, 18: 14342.5173, 20: 14342.5173, 22: 14342.5173, 24: 14342.5173, 26: 14342.5173, 28: 14342.5173, 30: 14342.5173, 32: 14342.5173, 34: 14342.5173, 36: 14342.5173, 38: 14342.5173, 40: 14342.5173, 42: 14342.5173, 44: 14342.5173, 46: 14342.5173, 48: 14342.5173, 50: 14342.5173, 52: 14342.5173, 54: 14342.5173, 56: 14342.5173, 58: 14342.5173, 60: 14342.5173, 62: 14342.5173, 64: 14342.5173, 66: 14342.5173, 68: 14342.5173, 70: 14342.5173, 72: 14342.5173, 74: 14342.5173, 76: 14342.5173, 78: 14342.5173, 80: 14342.519, 82: 14342.5207, 84: 14342.5224, 86: 14342.5233, 88: 14342.5233, 90: 14342.5241, 92: 14342.5241, 94: 14342.5241, 96: 14342.5241, 98: 14342.5241, 100: 14342.5241, 102: 14342.5241, 104: 14342.5241, 106: 14342.5241, 108: 14342.5241, 110: 14342.5241, 112: 14342.5241, 11

{'metadata': {'samplingRate': 1, 'lifetime': 1309, 'numSamples': 2322},
 1: {'dram': {'zero-intervals': {39: 2,
    0: 408,
    1: 27,
    38: 1,
    9: 2,
    7: 2,
    2: 13,
    28: 2,
    24: 1,
    62: 1,
    48: 1,
    17: 1,
    4: 2,
    14: 1,
    15: 2,
    6: 3,
    5: 4,
    43: 1,
    10: 1,
    11: 2,
    80: 1,
    29: 1},
   'nonzero-energy-increase': {'avg': 0.0018497907949799562,
    'stdev': 0.001562314964580888}},
  'pkg': {'zero-intervals': {0: 1158, 1: 1},
   'nonzero-energy-increase': {'avg': 0.031201466781713393,
    'stdev': 0.0234280854493221}}}}

In [39]:

json.dumps(result)

'{"metadata": {"samplingRate": 1, "lifetime": 1309, "numSamples": 2322}, "1": {"dram": {"zero-intervals": {"39": 2, "0": 408, "1": 27, "38": 1, "9": 2, "7": 2, "2": 13, "28": 2, "24": 1, "62": 1, "48": 1, "17": 1, "4": 2, "14": 1, "15": 2, "6": 3, "5": 4, "43": 1, "10": 1, "11": 2, "80": 1, "29": 1}, "nonzero-energy-increase": {"avg": 0.0018497907949799562, "stdev": 0.001562314964580888}}, "pkg": {"zero-intervals": {"0": 1158, "1": 1}, "nonzero-energy-increase": {"avg": 0.031201466781713393, "stdev": 0.0234280854493221}}}}'

In [ ]:
#with open(f'{filename_base}.stats.json','w') as fh: fh.write(json.dumps(result))